In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred

ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)
ge_genus_stage = load.loadGEOverlappingTCMA("genus", includeStage=True)
# aak_ge_stage = load.loadGEWithClinical(includeStage=True)

In [ ]:

# print(aak_ge_stage["stage"].value_counts(dropna=False))
# d = pred.attachStageStatus(aak_ge_stage)
# x, y = pr.splitData(d, "stage", project="STAD")
# print(y.value_counts())

# print(pred.runCrossValidation(x, y, splits=3, model="MLGR"))

In [ ]:

data, files = load.loadAll(includeStage=True, sameSamples=True)

In [ ]:
# data[1]["stage"].value_counts(ascending=True).iloc[0]

In [ ]:
# tcma_genus_aak_ge = load.loadGEOverlappingTCMA("genus", includeStage=True)
# aak_ge = load.loadGEWithClinical(includeStage=True)
# stage_data = [aak_ge, tcma_genus_aak_ge]
pred.runExperiments(data[:], files[:], "stage")